# Exercício 3 - PSI3471
### Exercício feito individualmente
#### Henrique Schneider Marx 14578432

Começamos importando as bibliotecas necessárias.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Aqui definimos a função de ativação sigmoidal e sua derivada, bem como a função de perda MSE (erro quadrático médio).

In [2]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
    fx = sigmoid(x)
    return fx * (1 - fx)

def mse_loss(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

A classe `NeuralNetwork` é definida abaixo. Ela contém a inicialização dos pesos, o método de alimentação para frente (feedforward) e o método de treinamento (train).

In [3]:
class NeuralNetwork:
    def _init_(self, num_inputs, num_hidden, num_outputs):
        self.num_inputs = num_inputs
        self.num_hidden = num_hidden
        self.num_outputs = num_outputs
        self.weights_hidden = np.random.randn(num_inputs, num_hidden)
        self.bias_hidden = np.zeros(num_hidden)
        self.weights_output = np.random.randn(num_hidden, num_outputs)
        self.bias_output = np.zeros(num_outputs)
        # Initialize parameters for Adam optimizer
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8
        self.m_w_hidden = np.zeros_like(self.weights_hidden)
        self.v_w_hidden = np.zeros_like(self.weights_hidden)
        self.m_b_hidden = np.zeros_like(self.bias_hidden)
        self.v_b_hidden = np.zeros_like(self.bias_hidden)
        self.m_w_output = np.zeros_like(self.weights_output)
        self.v_w_output = np.zeros_like(self.weights_output)
        self.m_b_output = np.zeros_like(self.bias_output)
        self.v_b_output = np.zeros_like(self.bias_output)
        pass

    def feedforward(self, x):
        hidden = sigmoid(np.dot(x, self.weights_hidden) + self.bias_hidden)
        output = sigmoid(np.dot(hidden, self.weights_output) + self.bias_output)
        return output
        pass

    def train(self, data, all_y_trues, learn_rate, epochs):
        loss_history = []
        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # Forward pass
                hidden = sigmoid(np.dot(x, self.weights_hidden) + self.bias_hidden)
                output = sigmoid(np.dot(hidden, self.weights_output) + self.bias_output)
                y_pred = output

                # Calculate partial derivatives
                d_L_d_ypred = -2 * (y_true - y_pred)

                # Output layer
                d_ypred_d_w_output = hidden * deriv_sigmoid(output)
                d_ypred_d_b_output = deriv_sigmoid(output)
                d_ypred_d_hidden = self.weights_output * deriv_sigmoid(output)

                # Update output layer weights and biases using Adam optimizer
                # Compute gradients for weights and biases
                grad_w_output = d_ypred_d_w_output
                grad_b_output = d_ypred_d_b_output

                # Update moment estimates for weights
                self.m_w_output = self.beta1 * self.m_w_output + (1 - self.beta1) * grad_w_output
                self.v_w_output = self.beta2 * self.v_w_output + (1 - self.beta2) * (grad_w_output ** 2)

                # Compute bias-corrected moment estimates for weights
                m_w_output_hat = self.m_w_output / (1 - self.beta1 ** (epoch + 1))
                v_w_output_hat = self.v_w_output / (1 - self.beta2 ** (epoch + 1))

                # Update weights
                self.weights_output -= learn_rate * m_w_output_hat / (np.sqrt(v_w_output_hat) + self.epsilon)

                # Update moment estimates for biases
                self.m_b_output = self.beta1 * self.m_b_output + (1 - self.beta1) * grad_b_output
                self.v_b_output = self.beta2 * self.v_b_output + (1 - self.beta2) * (grad_b_output ** 2)

                # Compute bias-corrected moment estimates for biases
                m_b_output_hat = self.m_b_output / (1 - self.beta1 ** (epoch + 1))
                v_b_output_hat = self.v_b_output / (1 - self.beta2 ** (epoch + 1))

                # Update biases
                self.bias_output -= learn_rate * m_b_output_hat / (np.sqrt(v_b_output_hat) + self.epsilon)

                # Hidden layer
                d_hidden_d_w_hidden = x * deriv_sigmoid(hidden).reshape(self.num_hidden, 1)

                # Update hidden layer weights and biases using Adam optimizer
                # Compute gradients for weights and biases
                grad_w_hidden = d_hidden_d_w_hidden
                grad_b_hidden = deriv_sigmoid(hidden)

                # Update moment estimates for weights
                self.m_w_hidden = self.beta1 * self.m_w_hidden + (1 - self.beta1) * grad_w_hidden
                self.v_w_hidden = self.beta2 * self.v_w_hidden + (1 - self.beta2) * (grad_w_hidden ** 2)

                # Compute bias-corrected moment estimates for weights
                m_w_hidden_hat = self.m_w_hidden / (1 - self.beta1 ** (epoch + 1))
                v_w_hidden_hat = self.v_w_hidden / (1 - self.beta2 ** (epoch + 1))

                # Update weights
                self.weights_hidden -= learn_rate * m_w_hidden_hat / (np.sqrt(v_w_hidden_hat) + self.epsilon)

                # Update moment estimates for biases
                self.m_b_hidden = self.beta1 * self.m_b_hidden + (1 - self.beta1) * grad_b_hidden
                self.v_b_hidden = self.beta2 * self.v_b_hidden + (1 - self.beta2) * (grad_b_hidden ** 2)

                # Compute bias-corrected moment estimates for biases
                m_b_hidden_hat = self.m_b_hidden / (1 - self.beta1 ** (epoch + 1))
                v_b_hidden_hat = self.v_b_hidden / (1 - self.beta2 ** (epoch + 1))

                # Update biases
                self.bias_hidden -= learn_rate * m_b_hidden_hat / (np.sqrt(v_b_hidden_hat) + self.epsilon)

            # Calculate loss for each epoch
            y_preds = np.apply_along_axis(self.feedforward, 1, data)
            loss = mse_loss(all_y_trues, y_preds)
            loss_history.append(loss)
            if epoch % 100 == 0:
                print("Epoch %d loss: %.3f" % (epoch, loss))
        pass

        return loss_history

Definimos as funções matemáticas que queremos que nossa rede neural aprenda a modelar.

In [4]:
def f1(x):
    return 1 / x

def f2(x):
    return np.log10(x)

def f3(x):
    return np.exp(-x)

def f4(x):
    return np.sin(x)

Preparamos os conjuntos de dados de treinamento e teste para cada função matemática.

In [5]:
x_train = np.linspace(1, 100, 10000).reshape(-1, 1)
y_train_f1 = f1(x_train)
y_train_f2 = f2(x_train[:1000])
y_train_f3 = f3(x_train[:1000])
y_train_f4 = f4(np.linspace(0, np.pi/2, 1000)).reshape(-1, 1)

x_test = np.linspace(1, 100, 1000).reshape(-1, 1)
y_test_f1 = f1(x_test)
y_test_f2 = f2(x_test[:100])
y_test_f3 = f3(x_test[:100])
y_test_f4 = f4(np.linspace(0, np.pi/2, 100)).reshape(-1, 1)

Aqui, treinamos a rede neural com diferentes números de neurônios na camada oculta e avaliamos seu desempenho.

In [6]:
hidden_neurons = [3, 4, 5, 10, 15, 20, 50, 100]

results = {}
for neurons in hidden_neurons:
    results[neurons] = {}
    for func, y_train, y_test in zip(['f1', 'f2', 'f3', 'f4'], [y_train_f1, y_train_f2, y_train_f3, y_train_f4], [y_test_f1, y_test_f2, y_test_f3, y_test_f4]):
        network = NeuralNetwork(num_inputs=1, num_hidden=neurons, num_outputs=1)
        loss_history = network.train(x_train, y_train, learn_rate=0.1, epochs=1000)
        y_pred = network.feedforward(x_test)
        mse = mse_loss(y_test, y_pred)
        results[neurons][func] = mse
        plt.plot(x_test, y_test, label='Real')
        plt.plot(x_test, y_pred, label='NN Prediction')
        plt.title(f'Function {func} - Hidden Neurons: {neurons}')
        plt.xlabel('x')
        plt.ylabel('f(x)')
        plt.legend()
        plt.show()

TypeError: NeuralNetwork() takes no arguments

Finalmente, apresentamos os resultados do treinamento, incluindo gráficos das aproximações da rede neural e uma tabela com os valores de MSE.

In [8]:
for neurons in hidden_neurons:
    print(f'Hidden Neurons: {neurons}')
    for func in ['f1', 'f2', 'f3', 'f4']:
        print(f'Function {func} MSE: {results[neurons][func]}')

Hidden Neurons: 3


KeyError: 'f1'